In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 14 14:48:40 2017

@author: rahul
"""

""" Find out the most similar words in the data.
This technique can help to analyze closest relationship among words, thus help identify the
type of accident assocated with word. Try out with different list of words and check the result. """

import re
import pandas as pd
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

import numpy as np
import string
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
'''Make possible for python notebooks to import the util as module. This needs to be copied at every python notebook which wants to 
load the module'''

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [3]:
# Read Files_osha
import tm_assignment_util as util
myutilObj = util.util()
Osha_AccidentCases = util.accidentCases_Osha

importing Jupyter notebook from tm_assignment_util.ipynb
['catch', 'machine', 'accident', 'inspect', 'maintain', 'excavator', 'magnet', 'machine', 'maintenance', 'carry', 'make', 'jump', 'grappler', 'turn', 'excavator', 'engine', 'grappler', 'spin', 'pin', 'grappler', 'excavator']


In [5]:
Osha_AccidentCases['Title_Summary_Case'] = Osha_AccidentCases['Title_Summary_Case']
#.apply(myutilObj.my_tokenizer)
Osha_AccidentCases.head()

,Cause,Title_Summary_Case,Title,Summary,Unnamed: 4,Unnamed: 5
0,Exposure to extreme temperatures,Employee Is Burned By Forklift Radiator Fluid...,Employee Is Burned By Forklift Radiator Fluid,At approximately 11:30 a.m. on November 13 2...,burn industrial truck waste proc fac pa...,
1,Falls,Employee Falls From Flatbed Trailer And Later...,Employee Falls From Flatbed Trailer And Later...,On August 30 2013 Employee #1 was working f...,truck flatbed truck trailer fall abdomen,
2,Caught in/between Objects,Employee Is Struck By Bales Of Wire And Kille...,Employee Is Struck By Bales Of Wire And Killed,On August 26 2013 Employee #1 with Lee Iro...,waste proc fac industrial truck struck b...,
3,Exposure to extreme temperatures,Employee Is Splashed With Hot Water And Is Bu...,Employee Is Splashed With Hot Water And Is Bu...,On July 14 2013 Employee #1 vacuum pump tr...,truck driver pump tank hot water struc...,
4,Exposure to extreme temperatures,Employee Suffers Burns While Moving Soup . O...,Employee Suffers Burns While Moving Soup,On June 30 2013 Employee #1 was working in ...,burn spill arm chest abdomen,


In [6]:
print(Osha_AccidentCases['Title_Summary_Case'])

0         Employee Is Burned By Forklift Radiator Fluid...
1         Employee Falls From Flatbed Trailer And Later...
2         Employee Is Struck By Bales Of Wire And Kille...
3         Employee Is Splashed With Hot Water And Is Bu...
4         Employee Suffers Burns While Moving Soup .  O...
5         Employee Injures Self With Knife .  An incide...
6         Foreman Is Fatally Crushed When Forklift Tips...
7         Employee Fractures Abdomen When Run Over By T...
8         Carpenter Injured In Abdomen When Saw Kicks B...
9         Employee Abdomen And Head Injury In Slip And ...
10        Employee Sustains Ruptured Spleen After Board...
11        Logger Is Injured When Butt Of Tree Kicks Out...
12        Worker Is Injured When Struck By Board And Th...
13        Employee Suffers Burns When He Came In Contac...
14        Bulldozer Operator Is Crushed While Working O...
15        Employee'S Abdomen Is Penetrated By A Wood Pa...
16        Employee Fractures Ankle Unclogging Beet Harv.

In [7]:
#text_noun = myutilObj.tag_noun_func_words(Osha_AccidentCases['Title_Summary_Case'])
text_noun = Osha_AccidentCases['Title_Summary_Case'].apply(myutilObj.tag_noun_func_words)

In [8]:
print(text_noun)

0        [Employee, Is, Forklift, Radiator, Fluid, Nove...
1        [Employee, Falls, From, Flatbed, Trailer, Late...
2        [Employee, Is, Struck, Bales, Wire, Killed, Au...
3        [Employee, Hot, Water, Is, Burned, July, Emplo...
4        [Employee, Suffers, Burns, Soup, June, Employe...
5        [Employee, Self, Knife, incident, Employee, pi...
6        [Foreman, Forklift, Tips, Over, May, Employee,...
7        [Employee, Fractures, Abdomen, Run, Tug, Cart,...
8        [Carpenter, Injured, Abdomen, Saw, Kicks, Back...
9        [Employee, Abdomen, Head, Injury, Slip, Fall, ...
10       [Employee, Sustains, Spleen, Board, Kicks, Bac...
11       [Logger, Butt, Tree, Kicks, Out, Strikes, Him,...
12       [Worker, Struck, Board, Thrown, Onto, Steps, I...
13       [Employee, Suffers, Burns, Came, Contact, Live...
14       [Bulldozer, Operator, Is, Crushed, Engine, Sep...
15       [Employee, Abdomen, Is, Wood, Pallet, a.m., Se...
16       [Employee, Fractures, Ankle, Unclogging, Beet,.

In [ ]:
"""def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 1. Remove non-letters
    sentence_text = re.sub(r'[^\w\s]','', sentence)
    # 2. Convert words to lower case and split them
    words = sentence_text.lower().split()
    # 3. Return a list of words
    return(words)"""

In [ ]:
"""def masia_to_sentences(masia, tokenizer, remove_stopwords=False ):
    try:
        # 1. Use the NLTK tokenizer to split the text into sentences
        raw_sentences = tokenizer.tokenize(masia.strip())
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call sentence_to_wordlist to get a list of words
                sentences.append(sentence_to_wordlist(raw_sentence))
        # 3. Return the list of sentences (each sentence is a list of words, so this returns a list of lists)
        len(sentences)
        return sentences
    except:
        print('nope')"""

In [ ]:
#nyt_masias = text_noun.tolist()
"""nyt_masias = Osha_AccidentCases['Title_Summary_Case'].tolist()
sentences = []

for i in range(0,len(nyt_masias)):
    try:
        # Need to first change "./." to "." so that sentences parse correctly
        masia = nyt_masias[i].replace("/.", '')
        # Now apply functions
        sentences += masia_to_sentences(masia, tokenizer)
    except:
        print('no!')

print("There are " + str(len(sentences)) + " sentences in our corpus of osha.")

#Here is what one sentence list looks like:
sentences[100]"""

In [9]:
# Word2Vec Implementation

import gensim
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models import Phrases
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

num_features = 300    # Word vector dimensionality 
# Try and change min word count to see different results                     
min_word_count = 1   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

model = word2vec.Word2Vec(text_noun, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, sg = 1) #skip gram sg =1

#If you don’t plan to train the model any further, calling init_sims will make the model will be better for memory:
model.init_sims(replace=True)

#Gensim makes it easy to store and load models:
model_name = "osha_accidents"
model.save(model_name)
new_model = gensim.models.Word2Vec.load('osha_accidents')


C:\ProgramFiles\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-10-25 20:51:03,978 : INFO : collecting all words and their counts
2017-10-25 20:51:03,979 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-25 20:51:04,113 : INFO : PROGRESS: at sentence #10000, processed 425185 words, keeping 25325 word types
2017-10-25 20:51:04,142 : INFO : collected 28707 word types from a corpus of 542357 raw words and 12830 sentences
2017-10-25 20:51:04,144 : INFO : Loading a fresh vocabulary
2017-10-25 20:51:04,308 : INFO : min_count=1 retains 28707 unique words (100% of original 28707, drops 0)
2017-10-25 20:51:04,309 : INFO : min_count=1 leaves 542357 word corpus (100% of original 542357, drops 0)
2017-10-25 20:51:04,512 : INFO : deleting the raw counts dictionary of 28707 items
2017-10-25 20:51:04,515 : INFO : sample=0.

In [10]:
#Investigate the semantic space
vocab = list(model.wv.vocab.keys())
vocab[:25]

['Employee',
 'Is',
 'Forklift',
 'Radiator',
 'Fluid',
 'November',
 'Edco',
 'Waste',
 'Recycling',
 'Services',
 'forklift',
 'Linde',
 'Lift',
 'Truck',
 'Number',
 'H2X393S04578',
 'employer',
 'FL-3',
 'bales',
 'paper',
 'products',
 'collection',
 'area',
 'yard',
 'trucks']

In [11]:
# Check if the word 'fell' exists in the vocabulary
'plumber' in model.wv.vocab

True

In [15]:

model.most_similar('hand',  topn=100)

#The semantic vector for ‘fell’ looks like this
#model['fell']

[('glove', 0.8870490789413452),
 ('tips', 0.8831332921981812),
 ('index', 0.8823497295379639),
 ('fingers', 0.87330162525177),
 ('spinning', 0.8721904158592224),
 ('flywheel', 0.8700660467147827),
 ('resistance', 0.8688862919807434),
 ('mechanisms', 0.867301344871521),
 ('cycling', 0.8667834401130676),
 ('amputations', 0.8647851943969727),
 ('spatula', 0.864740252494812),
 ('middle', 0.8634259104728699),
 ('nip', 0.8615032434463501),
 ('blades', 0.8612784743309021),
 ('knuckles', 0.8611034154891968),
 ('worm', 0.8608570098876953),
 ('groove', 0.8603649735450745),
 ('airlock', 0.8595898747444153),
 ('brushes', 0.859454870223999),
 ('sanding', 0.8594293594360352),
 ('platen', 0.8590328693389893),
 ('pinch', 0.8590306043624878),
 ('revolving', 0.8578372001647949),
 ('feeding', 0.8563242554664612),
 ('chopping', 0.855218768119812),
 ('catching', 0.8550158739089966),
 ('peddle', 0.8541698455810547),
 ('towel', 0.8540201783180237),
 ('knife', 0.853449821472168),
 ('shroud', 0.853264212608337

In [ ]:
# Earlier we used unigrams, now using bigrams
#bigramer = gensim.models.Phrases(sentences)
#model = Word2Vec(bigramer[sentences], workers=num_workers, \
#            size=num_features, min_count = min_word_count, \
#            window = context, sample = downsampling)

In [ ]:
#This can be done repeatedly to get tri-gram phrases (e.g. new_york_times)…but we’ll hold off. Here is the command incase you are interested
#trigram = Phrases(bigram[sentence_stream])

# semantic space looks like now that we’ve included bigrams
#vocab = list(model.wv.vocab.keys())

#vocab[:25]

In [ ]:
# assessing the relationship of words in our semantic space. Which words are  most similar
#model.most_similar('fell',  topn=15)

In [ ]:
#Do these terms differ if we exclude a relationship between +ve and -ve
#model.most_similar(positive=['start'], negative=['floor'], topn=15)

In [ ]:
# Word wiretap
#model.most_similar('engine',  topn=15)

In [ ]:
# word
#model.most_similar('worker',  topn=15)

In [ ]:
#What about the word ‘good’ without the word ‘bad’?
#model.most_similar(positive=['turn'], negative=['pinned'], topn=15)

#In case you want to add new sentences:
#bigram.add_vocab(new_sentence_stream)